In [ ]:
# based on https://github.com/happy-jihye/FFHQ-Alignment/blob/master/Anime-Face-Alignment
# https://github.com/hysts/anime-face-detector/blob/main/demo.ipynb

!sudo apt install ffmpeg
!pip install face-alignment
!pip install opencv-python
!git clone https://github.com/NVlabs/stylegan3.git

# anime-face-detector 
!pip install openmim
!mim install mmcv-full mmdet mmpose -y
!pip install anime-face-detector --no-dependencies


In [ ]:
import os
import sys
from glob import glob
from pathlib import Path
from tqdm import tqdm
import numpy as np
import scipy.ndimage
import cv2
import PIL
from PIL import Image
from shutil import copyfile
from IPython.display import display
import face_alignment
import anime_face_detector

videoDir = "Arcane"
frameDir = "frames"
alignedDir = "alignedFace"
filteredDir = "filteredFace"
preprocessedDir = "preprocessedFace"
dataZip= "arcaneFilteredData.zip"

for i in [videoDir,frameDir,alignedDir,filteredDir,preprocesseddDir]:
    os.makedirs(i, exist_ok=True)

In [ ]:
# # get frames from video
# videoList=glob(videoDir+"/*.mp4")

# # get 2 frame per sec, best jpg quality 
# for file in videoList:
#     name=Path(file).stem
#     !ffmpeg -i "$file" -r 2 -q:v 1 -qmin 1 -qmax 1 "$frameDir"/"$name"_%04d.jpg
# PIL.Image.open(glob(frameDir+"/*.jpg")[4])

In [ ]:
def image_align(src_file, dst_file, face_landmarks, output_size=256, transform_size=1024, enable_padding=True, use_landmark_28=False):
    # Align function from FFHQ dataset pre-processing step
    # https://github.com/NVlabs/ffhq-dataset/blob/master/download_ffhq.py


    if(use_landmark_28==False):
        lm = np.array(face_landmarks)
        lm_chin          = lm[0  : 17, :2]  # left-right
        lm_eyebrow_left  = lm[17 : 22, :2]  # left-right
        lm_eyebrow_right = lm[22 : 27, :2]  # left-right
        lm_nose          = lm[27 : 31, :2]  # top-down
        lm_nostrils      = lm[31 : 36, :2]  # top-down
        lm_eye_left      = lm[36 : 42, :2]  # left-clockwise
        lm_eye_right     = lm[42 : 48, :2]  # left-clockwise
        lm_mouth_outer   = lm[48 : 60, :2]  # left-clockwise
        lm_mouth_inner   = lm[60 : 68, :2]  # left-clockwise
        mouth_left   = lm_mouth_outer[0]
        mouth_right  = lm_mouth_outer[6]
        
    else:
        lm = np.array(face_landmarks)
        lm_eye_left      = lm[11 : 17, :2]  # left-clockwise
        lm_eye_right     = lm[17 : 23, :2]  # left-clockwise
        mouth_left   = lm[24, :2]
        mouth_right  = lm[26, :2]


    # Calculate auxiliary vectors.
    eye_left     = np.mean(lm_eye_left, axis=0)
    eye_right    = np.mean(lm_eye_right, axis=0)
    eye_avg      = (eye_left + eye_right) * 0.5
    eye_to_eye   = eye_right - eye_left
    mouth_avg    = (mouth_left + mouth_right) * 0.5
    eye_to_mouth = mouth_avg - eye_avg
    
    # Choose oriented crop rectangle.
    x = eye_to_eye - np.flipud(eye_to_mouth) * [-1, 1]
    x /= np.hypot(*x)
    x *= max(np.hypot(*eye_to_eye) * 2.0, np.hypot(*eye_to_mouth) * 1.8)
    y = np.flipud(x) * [-1, 1]
    c = eye_avg + eye_to_mouth * 0.1
    quad = np.stack([c - x - y, c - x + y, c + x + y, c + x - y])
    qsize = np.hypot(*x) * 2

    # Load image.
    img = PIL.Image.open(src_file)
    
    # Shrink.
    shrink = int(np.floor(qsize / output_size * 0.5))
    if shrink > 1:
        rsize = (int(np.rint(float(img.size[0]) / shrink)), int(np.rint(float(img.size[1]) / shrink)))
        img = img.resize(rsize, PIL.Image.ANTIALIAS)
        quad /= shrink
        qsize /= shrink
        
    # Crop.
    border = max(int(np.rint(qsize * 0.1)), 3)
    crop = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
    crop = (max(crop[0] - border, 0), max(crop[1] - border, 0), min(crop[2] + border, img.size[0]), min(crop[3] + border, img.size[1]))
    if crop[2] - crop[0] < img.size[0] or crop[3] - crop[1] < img.size[1]:
        img = img.crop(crop)
        quad -= crop[0:2]
        
    # Pad.
    pad = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
    pad = (max(-pad[0] + border, 0), max(-pad[1] + border, 0), max(pad[2] - img.size[0] + border, 0), max(pad[3] - img.size[1] + border, 0))
    if enable_padding and max(pad) > border - 4:
        pad = np.maximum(pad, int(np.rint(qsize * 0.3)))
        img = np.pad(np.float32(img), ((pad[1], pad[3]), (pad[0], pad[2]), (0, 0)), 'reflect')
        h, w, _ = img.shape
        y, x, _ = np.ogrid[:h, :w, :1]
        mask = np.maximum(1.0 - np.minimum(np.float32(x) / pad[0], np.float32(w-1-x) / pad[2]), 1.0 - np.minimum(np.float32(y) / pad[1], np.float32(h-1-y) / pad[3]))
        blur = qsize * 0.02
        img += (scipy.ndimage.gaussian_filter(img, [blur, blur, 0]) - img) * np.clip(mask * 3.0 + 1.0, 0.0, 1.0)
        img += (np.median(img, axis=(0,1)) - img) * np.clip(mask, 0.0, 1.0)
        img = PIL.Image.fromarray(np.uint8(np.clip(np.rint(img), 0, 255)), 'RGB')
        quad += pad[:2]
        
    # Transform.
    img = img.transform((transform_size, transform_size), PIL.Image.QUAD, (quad + 0.5).flatten(), PIL.Image.BILINEAR)
    if output_size < transform_size:
        img = img.resize((output_size, output_size), PIL.Image.ANTIALIAS)
     
     
    #display(img)
    # Save aligned image.
    img.save(dst_file, quality=100, subsampling=0)

landmarks_detector = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, flip_input=False)

In [ ]:
#get face image from frame
for frameFile in tqdm(glob(frameDir+"/*.jpg")):
    name=Path(frameFile).stem

    
    ######################## use anime face detector landmark to align
    
    image = cv2.imread(frameFile)
    preds = detector(image)
    for i, face_landmark in enumerate(preds):
        if face_landmark["bbox"][4]<0.5 or np.mean(face_landmark["keypoints"][:,2])<0.3: continue #skip low confidence

        aligned_face_path = os.path.join(alignedDir, name+"_"+str(i).zfill(4)+".jpg")
        image_align(frameFile, aligned_face_path, face_landmark["keypoints"],use_landmark_28=True)

    ######################## use face-alignment landmark to align
    
#     face_landmarks=landmarks_detector.get_landmarks(frameFile)
#     if face_landmarks is None: 
#         continue  #skip none output
#     for i, face_landmark in enumerate(face_landmarks):
#         aligned_face_path = os.path.join(alignedDir, name+"_"+str(i).zfill(4)+".jpg")
#         image_align(frameFile, aligned_face_path, face_landmark)
        
        

In [ ]:
#filter blurry image
threshold=70

for i,file in tqdm(enumerate(glob(alignedDir+"/*.jpg"))):
    name=Path(file).name
    image = cv2.imread(file)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # fm= cv2.Laplacian(gray, cv2.CV_64F).var()
    fm=np.max(cv2.convertScaleAbs(cv2.Laplacian(gray,3)))
    
    if threshold < fm:        
        # display(Image.open(file))
        copyfile(file, filteredDir+"/"+name)

In [ ]:
# color correction and denoise
def better_cb(img, percent=1):
    # from https://github.com/luftj/MaRE/blob/4284fe2b3307ca407e87e3b0dbdaa3c1ef646731/simple_cb.py

    if not percent or percent == 0 or percent == 100:
        return img

    out_channels = []
    cumstops = (
        img.shape[0] * img.shape[1] * percent / 200.0,
        img.shape[0] * img.shape[1] * (1 - percent / 200.0),
    )
    for channel in cv2.split(img):
        cumhist = np.cumsum(cv2.calcHist([channel], [0], None, [256], (0, 256)))
        low_cut, high_cut = np.searchsorted(cumhist, cumstops)
        lut = np.concatenate(
            (
                np.zeros(low_cut),
                np.around(np.linspace(0, 255, high_cut - low_cut + 1)),
                255 * np.ones(255 - high_cut),
            )
        )
        out_channels.append(cv2.LUT(channel, lut.astype("uint8")))
    return cv2.merge(out_channels)


for i, file in tqdm(enumerate(glob(filteredDir + "/*.jpg"))):
    name = Path(file).name
    image = cv2.imread(file)
    image = better_cb(image)  # color correction
    image = cv2.fastNlMeansDenoisingColored(image, None, 3, 3, 7, 21)  # denoise
    img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    img.save(preprocessedDir + "/" + name, quality=100, subsampling=0)

In [ ]:
#display
for i,file in tqdm(enumerate(glob(preprocesseddDir+"/*.jpg")[0:10])):
    display(Image.open(file))
print(len(glob(filteredDir+"/*.jpg")))

In [ ]:
# make zip file
!cd stylegan3 && python dataset_tool.py --source="../$preprocessedDir" --dest="../$dataZip"
